In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
from keras.optimizers import Adam
from data import *
from metrics import  dice,IU,recall,precision,f1,specificity
from losses import dice_loss
from modelDropOut import *

In [4]:
from losses import bce_dice_loss
optim = Adam(lr = 1e-4)
img_size_target = 384
modelFolder = 'SaveModels/final'
model = Unet( backbone_name='efficientnet-b5', input_shape=(img_size_target, img_size_target, 3),  encoder_freeze=False, encoder_weights='imagenet')
model.load_weights(os.path.join(modelFolder,'model_efficientnetb5_dice.h5'))
model.compile(optimizer=optim, loss=bce_dice_loss, metrics=['acc',dice,IU,recall,precision,f1,specificity])

Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.


In [ ]:
from keras.models import Model
from keras.layers import Activation
activation='sigmoid'
model.layers.pop()
model.trainable = False
from crfrnn_layer import CrfRnnLayer
crf_layer = CrfRnnLayer(image_dims=(img_size_target,img_size_target),
                            num_classes=1,
                            theta_alpha=160.,
                            theta_beta=3.,
                            theta_gamma=3.,
                            num_iterations=2,
                            name='crfrnn')([model.layers[-1].output, model.inputs[0]])
x = Activation(activation, name=activation)(crf_layer)
model_crf = Model(inputs = model.input,outputs = x)
model_crf.compile(optimizer=optim, loss=bce_dice_loss, metrics=['acc',dice,IU,recall,precision,f1,specificity])

In [ ]:
model_crf.summary()

In [ ]:
data_gen_args=dict()
data_gen_args = dict(
                    rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    brightness_range=[1, 1.5],
                    horizontal_flip=True,
                    vertical_flip=True,  
                    #zca_whitening=True,
                    #preprocessing_function = _augment,
                    fill_mode='nearest')
train_generator = trainGenerator(1,'data/train','image','label',data_gen_args,save_to_dir = None,image_size=img_size_target)


In [ ]:
data_gen_args=dict()
validation_generator = trainGenerator(1,'data/valid','image','label',data_gen_args,save_to_dir = None,image_size=img_size_target)

In [ ]:
from keras.callbacks import ModelCheckpoint
epochs = 1
batch_size = 1
steps_per_epoch = 100
validation_steps = 10
modelFolder = 'SaveModels/CRF'
filepathsave=os.path.join(modelFolder,'model_efficientnetb5_dice_crf.h5')
checkpoint = ModelCheckpoint(filepathsave, monitor='val_dice', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
history =model_crf.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,                              
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

In [ ]:
def testGenerator1(test_path,num_image = 28,target_size = (384,384),flag_multi_class = False,as_gray = False):
    sourceFiles1 = os.listdir(test_path)
    for i in range(len(sourceFiles1)):
        img = io.imread(os.path.join(test_path,sourceFiles1[i]),as_gray = as_gray)
        img = img / 255
        img = trans.resize(img,target_size)
        #img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img

In [ ]:
testGene = testGenerator1("data/CVC-ColonDB/Image")
results = model.predict_generator(testGene,300,verbose=0)

In [ ]:
from data import *
data_gen_args=dict()
Colon_generator = trainGenerator(1,'data/CVC-ColonDB','Image','label',data_gen_args,save_to_dir = None,image_size=img_size_target)

In [ ]:
from data import *
data_gen_args=dict()
ETIS_generator = trainGenerator(1,'data/ETIS-Larib','Image','label',data_gen_args,save_to_dir = None,image_size=img_size_target)

In [ ]:
scores_val = model_crf.evaluate_generator(Colon_generator,300, verbose=0)

In [ ]:
print("Score of:")
print("- val_acc:\t\t{}".format(scores_val[1]))
print("- val_dice:\t{}".format(scores_val[2]))
print("- val_iou:\t{}".format(scores_val[3]))
print("- val_recall:\t{}".format(scores_val[4]))
print("- val_precision:\t{}".format(scores_val[5]))
print("- val_f1:\t{}".format(scores_val[6]))
print("- specificity:\t{}".format(scores_val[7]))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
image_size=384
i=2
sourceFiles_label = os.listdir("data/CVC-ColonDB/label")
label_path="data/CVC-ColonDB/label"
img_label = io.imread(os.path.join(label_path,sourceFiles_label[i]),as_gray = True)
img_label = trans.resize(img_label,(image_size, image_size))
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax = fig.add_subplot(1, 2, 1)
ax.imshow(np.reshape(results[i]*255, (image_size, image_size)), cmap="gray")

ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(img_label, (image_size, image_size)), cmap="gray")

In [ ]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax = fig.add_subplot(1, 2, 1)
ax.imshow(np.reshape(results[i]*255, (image_size, image_size)), cmap="gray")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training acc')
plt.plot(val_acc, label='Validation acc')
plt.legend(loc='lower right')
plt.ylabel('acc')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation acc')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
acc = history.history['loss']
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training loss')
#plt.legend(loc='lower right')
#plt.ylabel('Training loss')
plt.ylim([min(plt.ylim()),1])
plt.title('Training loss')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
dice = history.history['dice']
val_dice = history.history['val_dice']
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(dice, label='Training dice')
plt.plot(val_dice, label='Validation dice')
plt.legend(loc='lower right')
plt.ylabel('dice')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation dice')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
IU = history.history['IU']
val_IU = history.history['val_IU']
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(IU, label='Training IU')
plt.plot(val_IU, label='Validation IU')
plt.legend(loc='lower right')
plt.ylabel('IU')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation IU')


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
recall = history.history['recall']
val_recall = history.history['val_recall']
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(recall, label='Training recall')
plt.plot(val_recall, label='Validation recall')
plt.legend(loc='lower right')
plt.ylabel('recall')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation recall')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
precision = history.history['precision']
val_precision = history.history['val_precision']
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(precision, label='Training precision')
plt.plot(val_precision, label='Validation precision')
plt.legend(loc='lower right')
plt.ylabel('precision')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation precision')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
f1 = history.history['f1']
val_f1 = history.history['val_f1']
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(f1, label='Training f1')
plt.plot(val_f1, label='Validation f1')
plt.legend(loc='lower right')
plt.ylabel('f1')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation f1')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
specificity = history.history['specificity']
val_specificity = history.history['val_specificity']
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(specificity, label='Training specificity')
plt.plot(val_specificity, label='Validation specificity')
plt.legend(loc='lower right')
plt.ylabel('specificity')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation specificity')